In [1]:
"""
Created on Tuesday 22 March 2022
Author: ZMW
"""
#import libraries
import pandas as pd
import numpy as np
import glob
import os
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

In [19]:
#define function
def func(x, a, b, c):  #Position as a function of time.
    return a*(2/np.pi)*np.arcsin(np.sin(np.pi*(b*x+c)))

In [20]:
# data path
path = [
        '../../smooth_pursuit/Eye_Tracker_Data/HC_1/',
        '../../smooth_pursuit/Eye_Tracker_Data/PD_1/'
        ]

In [21]:
#Columns names and hc subject code
columns_hc = [
              'Label',
              'Patient',
              'A1_1Deg','A1_1Deg_err','B1_1Deg','B1_1Deg_err','C1_1Deg','C1_1Deg_err','1_Deg_ResSoS_1','1Deg_AdjR2_1',
              'A2_1Deg','A2_1Deg_err','B2_1Deg','B2_1Deg_err','C2_1Deg','C2_1Deg_err','1_Deg_ResSoS_2','1Deg_AdjR2_2',

              'A1_2Deg','A1_2Deg_err','B1_2Deg','B1_2Deg_err','C1_2Deg','C1_2Deg_err','2_Deg_ResSoS_1','2Deg_AdjR2_1',
              'A2_2Deg','A2_2Deg_err','B2_2Deg','B2_2Deg_err','C2_2Deg','C2_2Deg_err','2_Deg_ResSoS_2','2Deg_AdjR2_2',

              'A1_4Deg','A1_4Deg_err','B1_4Deg','B1_4Deg_err','C1_4Deg','C1_4Deg_err','4_Deg_ResSoS_1','4Deg_AdjR2_1',
              'A2_4Deg','A2_4Deg_err','B2_4Deg','B2_4Deg_err','C2_4Deg','C2_4Deg_err','4_Deg_ResSoS_2','4Deg_AdjR2_2',

              'A1_6Deg','A1_6Deg_err','B1_6Deg','B1_6Deg_err','C1_6Deg','C1_6Deg_err','6_Deg_ResSoS_1','6Deg_AdjR2_1',
              'A2_6Deg','A2_6Deg_err','B2_6Deg','B2_6Deg_err','C2_6Deg','C2_6Deg_err','6_Deg_ResSoS_2','6Deg_AdjR2_2',

              'A1_8Deg','A1_8Deg_err','B1_8Deg','B1_8Deg_err','C1_8Deg','C1_8Deg_err','8_Deg_ResSoS_1','8Deg_AdjR2_1',
              'A2_8Deg','A2_8Deg_err','B2_8Deg','B2_8Deg_err','C2_8Deg','C2_8Deg_err','8_Deg_ResSoS_2','8Deg_AdjR2_2',

]

In [22]:
# create empty dataframe
fit = pd.DataFrame([],columns = columns_hc)
fit['Patient'] = [
          'PD001','PD002','PD003','PD004','PD005','PD006','PD007','PD008',
          'PD009','PD010','PD011','PD011','PD012','PD013',

          'PDPW102','PDPW106','PDPW107','PDPW110','PDPW111','PDPW112',
          'PDPY101','PDQE102','PDQE104','PDQE105','PDQE107','PDQE108',
          'PDQE112','PDQE113','PDQE115'

          ]
fit['Label'] =[
               "Not_Parkinson's","Not_Parkinson's","Not_Parkinson's","Not_Parkinson's",
               "Not_Parkinson's","Not_Parkinson's","Not_Parkinson's","Not_Parkinson's",
               "Not_Parkinson's","Not_Parkinson's","Not_Parkinson's","Not_Parkinson's",
               "Not_Parkinson's","Not_Parkinson's",


               "Parkinson's","Parkinson's","Parkinson's","Parkinson's","Parkinson's",
               "Parkinson's","Parkinson's","Parkinson's","Parkinson's","Parkinson's",
               "Parkinson's","Parkinson's","Parkinson's","Parkinson's","Parkinson's",
               
]

In [23]:
patient_name = [
          'PD001','PD002','PD003','PD004','PD005','PD006','PD007','PD008',
          'PD009','PD010','PD011','PD011','PD012','PD013',

          'PDPW102','PDPW106','PDPW107','PDPW110','PDPW111','PDPW112',
          'PDPY101','PDQE102','PDQE104','PDQE105','PDQE107','PDQE108',
          'PDQE112','PDQE113','PDQE115'

          ]

In [24]:
# Fitting triangle wave function
file_hc = os.listdir(path[0])
for file in file_hc:
    if file == 'one_deg.csv':
        hc = pd.read_csv(path[0]+file)
        for column in hc.columns:
            hc[column] = hc[column].interpolate().ffill().bfill()
        for column,content in hc.iteritems():
            for subj in patient_name:
                if str(subj) in column:
                    if 'x1' in column:
                        time = hc[column]
                    elif 'y1' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.05, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[2:10]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
                    elif 'x2' in column:
                        time = hc[column]
                    
                    elif 'y2' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.05, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[10:18]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
    
    elif file == 'two_deg.csv':
        hc = pd.read_csv(path[0]+file)
        for column in hc.columns:
            hc[column] = hc[column].interpolate().ffill().bfill()
        for column,content in hc.iteritems():
            for subj in patient_name:
                if str(subj) in column:
                    if 'x1' in column:
                        time = hc[column]
                    elif 'y1' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.1, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[18:26]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
                    elif 'x2' in column:
                        time = hc[column]
                    
                    elif 'y2' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.1, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[26:34]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]

    elif file == 'four_deg.csv':
        hc = pd.read_csv(path[0]+file)
        for column in hc.columns:
            hc[column] = hc[column].interpolate().ffill().bfill()
        for column,content in hc.iteritems():
            for subj in patient_name:
                if str(subj) in column:
                    if 'x1' in column:
                        time = hc[column]
                    elif 'y1' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.2, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[34:42]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
                    elif 'x2' in column:
                        time = hc[column]
                    
                    elif 'y2' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.2, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[42:50]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]             

    elif file == 'six_deg.csv':
        hc = pd.read_csv(path[0]+file)
        for column in hc.columns:
            hc[column] = hc[column].interpolate().ffill().bfill()
        for column,content in hc.iteritems():
            for subj in patient_name:
                if str(subj) in column:
                    if 'x1' in column:
                        time = hc[column]
                    elif 'y1' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.3, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[50:58]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
                    elif 'x2' in column:
                        time = hc[column]
                    
                    elif 'y2' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.3, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[58:66]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
    
    elif file == 'eight_deg.csv':
        hc = pd.read_csv(path[0]+file)
        for column in hc.columns:
            hc[column] = hc[column].interpolate().ffill().bfill()
        for column,content in hc.iteritems():
            for subj in patient_name:
                if str(subj) in column:
                    if 'x1' in column:
                        time = hc[column]
                    elif 'y1' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.4, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[66:74]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
                    elif 'x2' in column:
                        time = hc[column]
                    
                    elif 'y2' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.4, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[74:82]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]


file_pd = os.listdir(path[1])
for file in file_pd:
    if file == 'one_deg.csv':
        hc = pd.read_csv(path[1]+file)
        for column in hc.columns:
            hc[column] = hc[column].interpolate().ffill().bfill()
        for column,content in hc.iteritems():
            for subj in patient_name:
                if str(subj) in column:
                    if 'x1' in column:
                        time = hc[column]
                    elif 'y1' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.05, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[2:10]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
                    elif 'x2' in column:
                        time = hc[column]
                    
                    elif 'y2' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.05, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[10:18]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
    
    elif file == 'two_deg.csv':
        hc = pd.read_csv(path[1]+file)
        for column in hc.columns:
            hc[column] = hc[column].interpolate().ffill().bfill()
        for column,content in hc.iteritems():
            for subj in patient_name:
                if str(subj) in column:
                    if 'x1' in column:
                        time = hc[column]
                    elif 'y1' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.1, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[18:26]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
                    elif 'x2' in column:
                        time = hc[column]
                    
                    elif 'y2' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.1, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[26:34]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]

    elif file == 'four_deg.csv':
        hc = pd.read_csv(path[1]+file)
        for column in hc.columns:
            hc[column] = hc[column].interpolate().ffill().bfill()
        for column,content in hc.iteritems():
            for subj in patient_name:
                if str(subj) in column:
                    if 'x1' in column:
                        time = hc[column]
                    elif 'y1' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.2, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[34:42]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
                    elif 'x2' in column:
                        time = hc[column]
                    
                    elif 'y2' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.2, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[42:50]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]             

    elif file == 'six_deg.csv':
        hc = pd.read_csv(path[1]+file)
        for column in hc.columns:
            hc[column] = hc[column].interpolate().ffill().bfill()
        for column,content in hc.iteritems():
            for subj in patient_name:
                if str(subj) in column:
                    if 'x1' in column:
                        time = hc[column]
                    elif 'y1' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.3, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[50:58]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
                    elif 'x2' in column:
                        time = hc[column]
                    
                    elif 'y2' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.3, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[58:66]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
    
    elif file == 'eight_deg.csv':
        hc = pd.read_csv(path[1]+file)
        for column in hc.columns:
            hc[column] = hc[column].interpolate().ffill().bfill()
        for column,content in hc.iteritems():
            for subj in patient_name:
                if str(subj) in column:
                    if 'x1' in column:
                        time = hc[column]
                    elif 'y1' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.4, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[66:74]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]
                    elif 'x2' in column:
                        time = hc[column]
                    
                    elif 'y2' in column:
                        position = hc [column]
                        ET_popt, ET_pcov = curve_fit(func, time, position, p0=[10, 0.4, 0])
                        ET_perr = np.sqrt(np.diag(ET_pcov))
                        Y_pred = func(time, *ET_popt)
                        R_sq_ET = r2_score(Y_pred, position)
                        ResSoS = np.sum(np.square(Y_pred-position))
                        fit.loc[fit['Patient']== str(subj),fit.columns[74:82]] = [ET_popt[0],
                                                    ET_perr[0],ET_popt[1],ET_perr[1],ET_popt[2],ET_perr[2],ResSoS,R_sq_ET]




In [25]:
fit

,Label,Patient,A1_1Deg,A1_1Deg_err,B1_1Deg,B1_1Deg_err,C1_1Deg,C1_1Deg_err,1_Deg_ResSoS_1,1Deg_AdjR2_1,...,8_Deg_ResSoS_1,8Deg_AdjR2_1,A2_8Deg,A2_8Deg_err,B2_8Deg,B2_8Deg_err,C2_8Deg,C2_8Deg_err,8_Deg_ResSoS_2,8Deg_AdjR2_2
0,Not_Parkinson's,PD001,10.38819,0.006132,0.049329,0.000006,0.011989,0.000337,8928.709194,0.990598,...,6698.838281,0.951413,10.273124,0.044206,0.386347,0.000384,0.037356,0.002439,6809.4558,0.939095
1,Not_Parkinson's,PD002,10.429055,0.006929,0.049478,0.000007,0.002837,0.000379,11388.604704,0.988092,...,4769.901822,0.96736,11.4825,0.049154,0.395412,0.00038,-0.002486,0.002419,8383.48184,0.939586
2,Not_Parkinson's,PD003,10.097088,0.005091,0.049368,0.000006,0.005707,0.000287,6140.292312,0.993141,...,3488.747176,0.968709,10.084338,0.019838,0.394595,0.000174,-0.035466,0.001106,1349.96224,0.987259
3,Not_Parkinson's,PD004,10.482881,0.022179,0.049585,0.000023,0.016056,0.001212,117748.97971,0.879372,...,10972.045844,0.920137,11.493414,0.048867,0.401386,0.000377,-0.042463,0.002395,8242.136178,0.94037
4,Not_Parkinson's,PD005,10.181273,0.008833,0.049368,0.00001,0.001569,0.000494,18447.340517,0.979692,...,18268.960666,0.809586,10.370348,0.036121,0.395392,0.000308,-0.030769,0.00196,4485.300264,0.960035
5,Not_Parkinson's,PD006,10.302529,0.01045,0.049283,0.000011,0.019735,0.00058,25994.527943,0.972249,...,7175.782112,0.938474,10.356903,0.067627,0.410282,0.000584,-0.052723,0.003707,16046.923323,0.859535
6,Not_Parkinson's,PD007,9.934379,0.007838,0.049209,0.000009,0.003633,0.000449,14490.496516,0.9832,...,1465.027123,0.985896,9.801698,0.024067,0.39785,0.000217,-0.045138,0.00138,1986.965945,0.980134
7,Not_Parkinson's,PD008,9.861257,0.026999,0.04922,0.00003,0.051865,0.001576,176009.839073,0.798036,...,15508.422642,0.870504,8.591437,0.105215,0.415729,0.001098,-0.078941,0.006967,39005.324498,0.507026
8,Not_Parkinson's,PD009,10.221777,0.005149,0.049244,0.000006,0.002274,0.000287,6253.730577,0.993152,...,849.60321,0.992275,10.305625,0.022845,0.396998,0.000196,-0.056233,0.001244,1783.111545,0.983816
9,Not_Parkinson's,PD010,10.244784,0.017579,0.050004,0.000019,-0.015054,0.000981,73606.211066,0.920684,...,13818.78756,0.878166,10.99796,0.085181,0.406918,0.00069,-0.043189,0.004387,25335.919452,0.802179


In [26]:
fit.to_csv('data.csv',index=False)

In [281]:
test = pd.read_csv(path[0]+'four_deg.csv')
test.isna().sum()

x1_PD009_right     0
y1_PD009_right     0
x2_PD009_right     0
y2_PD009_right     0
x1_PD012_left      0
y1_PD012_left      0
x2_PD012_left      0
y2_PD012_left      0
x1_PD004_left      0
y1_PD004_left      0
x2_PD004_left      0
y2_PD004_left      0
x1_PD013_right     0
y1_PD013_right     0
x2_PD013_right     0
y2_PD013_right     0
x1_PD002_left      0
y1_PD002_left      0
x2_PD002_left      0
y2_PD002_left      0
x1_PD008_right     0
y1_PD008_right     0
x2_PD008_right    70
y2_PD008_right    70
x1_PD011_right     0
y1_PD011_right     0
x2_PD011_right     1
y2_PD011_right     1
x1_PD003_left      0
y1_PD003_left      0
x2_PD003_left      0
y2_PD003_left      0
x1_PD001_right     0
y1_PD001_right     0
x2_PD001_right     0
y2_PD001_right     0
x1_PD006_left      0
y1_PD006_left      0
x2_PD006_left      0
y2_PD006_left      0
x1_PD010_left      0
y1_PD010_left      0
x2_PD010_left      0
y2_PD010_left      0
x1_PD007_right     0
y1_PD007_right     0
x2_PD007_right     0
y2_PD007_righ

In [254]:
fit.columns[9:17]

Index(['A2_1Deg', 'A2_1Deg_err', 'B2_1Deg', 'B2_1Deg_err', 'C2_1Deg',
       'C2_1Deg_err', '1_Deg_ResSoS_2', '1Deg_AdjR2_2'],
      dtype='object')

In [256]:
fit

,Patient,A1_1Deg,A1_1Deg_err,B1_1Deg,B1_1Deg_err,C1_1Deg,C1_1Deg_err,1_Deg_ResSoS_1,1Deg_AdjR2_1,A2_1Deg,...,8_Deg_ResSoS_1,8Deg_AdjR2_1,A2_8Deg,A2_8Deg_err,B2_8Deg,B2_8Deg_err,C2_8Deg,C2_8Deg_err,8_Deg_ResSoS_2,8Deg_AdjR2_2
0,PD001,10.38819,0.006132,0.049329,0.000006,0.011989,0.000337,8928.709194,0.990598,10.379248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PD002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PD003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PD004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PD005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PD006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,PD007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,PD008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,PD009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,PD010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
name = ['a','b']
print(name)

['a', 'b']


In [135]:
fit

,Patient,A1_1Deg,A1_1Deg_err,B1_1Deg,B1_1Deg_err,C1_1Deg,C1_1Deg_err,1_Deg_ResSoS_1,1Deg_AdjR2_1,A2_1Deg,...,8_Deg_ResSoS_1,8Deg_AdjR2_1,A2_8Deg,A2_8Deg_err,B2_8Deg,B2_8Deg_err,C2_8Deg,C2_8Deg_err,8_Deg_ResSoS_2,8Deg_AdjR2_2
0,PD001,zaw,myo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PD002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PD003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PD004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PD005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PD006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,PD007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,PD008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,PD009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,PD010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
